In [164]:
import time
import rasterio
import numpy as np
import math
from pathlib import Path 
from osgeo import gdal
import glob
import geopandas as gpd
import pandas as pd
import sys

# 0. Data Check
## treepoint data list
- manup_treepoint.shp
- mandown_treepoint.shp
- br_rest1_treepoint.shp
- br_rest2_treepoint.shp

In [165]:
tp_up = gpd.GeoDataFrame.from_file("tree_points/manup_treepoint.shp")
tp_down = gpd.GeoDataFrame.from_file("tree_points/mandown_treepoint.shp")
tp_br1 = gpd.GeoDataFrame.from_file("tree_points/br_rest1_treepoint.shp")
tp_br2 = gpd.GeoDataFrame.from_file("tree_points/br_rest2_treepoint.shp")
tpall = gpd.GeoDataFrame( pd.concat( [tp_up, tp_down, tp_br1, tp_br2], ignore_index=True) )

In [166]:
# print(tp_up.info())
# print(tp_down.info())
# print(tp_br1.info())
# print(tp_br2.info())
# print(tpall.info())

## subclass change
- tpall (up, down, br) 모든 데이터 합쳐서 시작
- up[0:28273], down[28273:52401], br1[52401:74639], br2[74639:85799] (*종료점은 포함x)

In [167]:
#### 그냥 일단 합쳐서 만들기
# replace with *
tpall_sc = tpall['subclass'].fillna('*')

# subclass string to int
tpall_sc = tpall_sc.to_frame() # 열 하나만 추출하면 series로 type 바뀌어서 다시 dataframe으로 바꿔줘야 함.
sc_list = {'subclass':{'Rosidae':'1', 'Hamamelididae':'2', 'Asteridae':'3',
                           'Dilleniidae':'4', 'Magnoliidae':'5', '*':'6'}}
tpall_sc = tpall_sc.replace(sc_list) # sc_list의 내용을 (tp_up_sc)에 대체해라.
print(f'{tpall_sc.shape} {tpall_sc}')
    
# change .. lat이랑 long을 넣냐 안넣냐..
tpall.drop(["subclass", "GenusSpeci", "OBJECTID", "Lat", "Long"], axis = 1, inplace=True)
tpall = pd.concat([tpall, tpall_sc], axis = 1) #tpall_sc 먼저 쓰면 type이 dataframe으로 바뀌어서 포인트 샘플링을 못함. tpall 먼저 써야함.
print(tpall.info())

(85799, 1)       subclass
0            1
1            2
2            1
3            2
4            2
...        ...
85794        1
85795        6
85796        2
85797        1
85798        3

[85799 rows x 1 columns]
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85799 entries, 0 to 85798
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  85799 non-null  geometry
 1   subclass  85799 non-null  object  
dtypes: geometry(1), object(1)
memory usage: 1.3+ MB
None


In [168]:
type(tpall)

geopandas.geodataframe.GeoDataFrame

# 1. Point Sampling

In [169]:
# file에 영상 넣기 # NDVIup_file, NDVIdown_file, NDVIbr_file
for i in ['NDVI', 'SBI', 'GVI', 'YVI', 'WBI']:
    for j in ['up', 'down', 'br']:
        globals()[f'{i}{j}_file'] = '../KOMPSAT 위성영상/데이터 전처리/4. data/'+j+'/'+i+'.tif'
        globals()[f'{i}{j}_src'] = rasterio.open(globals()[f'{i}{j}_file'])

In [170]:
## 반복변수명과 값 확인
# for i in globals():
#     if i[-4:] == 'file':
#         print(f'{i}: {globals()[i]}')
#     elif i[-3:] == 'src':
#         print(f'{i}: {globals()[i]}')
#     else:
#         pass

In [171]:
# 각 파일의 데이터 개수 세기
datalen = [len(tp_up), len(tp_down), len(tp_br1), len(tp_br2)]
print(datalen)

[28273, 24128, 22238, 11160]


In [172]:
# 파일별로 tpall dataframe에 값 입력하기 (NDVIup, NDVIdown, NDVIbr)
# up[0:28273], down[28273:52401], br1[52401:74639], br2[74639:85799] (*종료점은 포함x)

## 동적할당...이 필요한 것 같아서ㅜㅜ 강의 듣고 다시 해보든가 해야할 듯.. 일단은 또 복붙..!

# for i in ['up', 'down', 'br']:
#     tpall[Path(globals()[f'NDVI{i}_file']).stem] = [x for x in globals()[f'NDVI{i}_src'](coord_list)] # 포인트 샘플링
#     tpall[Path(globals()[f'NDVI{i}_file']).stem] = tpall[Path(globals()[f'NDVI{i}_file']).stem].astype('float64')

In [173]:
tpup = tpall[:datalen[0]]
tpdown = tpall[datalen[0]:datalen[0]+datalen[1]]
tpbr = tpall[datalen[0]+datalen[1]:]
tpbr

,geometry,subclass
52401,POINT (-73.92916 40.61451),2
52402,POINT (-73.93893 40.61036),3
52403,POINT (-73.92881 40.62688),2
52404,POINT (-73.92100 40.61747),1
52405,POINT (-73.92101 40.61761),1
...,...,...
85794,POINT (-73.89033 40.65515),1
85795,POINT (-73.90517 40.64284),6
85796,POINT (-73.90518 40.64321),2
85797,POINT (-73.89036 40.65520),1


In [174]:
# just NDVI point sampling  # 포인트의 좌표를 리스트로 추출
coord_uplist = [(x,y) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)] 
coord_downlist = [(x,y) for x,y in zip(tpdown['geometry'].x , tpdown['geometry'].y)] 
coord_brlist = [(x,y) for x,y in zip(tpbr['geometry'].x , tpbr['geometry'].y)] 
print(f'{len(coord_uplist)}, {len(coord_downlist)}, {len(coord_brlist)}')

28273, 24128, 33398


In [175]:
tpup[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_uplist)] # 포인트 샘플링
tpup[Path(NDVIup_file).stem] = tpup[Path(NDVIup_file).stem].astype('float64')

tpdown[Path(NDVIdown_file).stem] = [x for x in NDVIdown_src.sample(coord_downlist)] # 포인트 샘플링
tpdown[Path(NDVIdown_file).stem] = tpdown[Path(NDVIdown_file).stem].astype('float64')

tpbr[Path(NDVIbr_file).stem] = [x for x in NDVIbr_src.sample(coord_brlist)] # 포인트 샘플링
tpbr[Path(NDVIbr_file).stem] = tpbr[Path(NDVIbr_file).stem].astype('float64')

tpall = pd.concat([tpup, tpdown, tpbr])
tpall

C:\Users\admin\anaconda3\envs\sprout38\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\admin\anaconda3\envs\sprout38\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\admin\anaconda3\envs\sprout38\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,geometry,subclass,NDVI
0,POINT (-73.95018 40.80481),1,0.061114
1,POINT (-73.95173 40.80921),2,-0.068959
2,POINT (-73.94942 40.80449),1,-0.001804
3,POINT (-73.95062 40.80500),2,0.038803
4,POINT (-73.95038 40.80490),2,0.227972
...,...,...,...
85794,POINT (-73.89033 40.65515),1,0.092050
85795,POINT (-73.90517 40.64284),6,0.148645
85796,POINT (-73.90518 40.64321),2,0.002555
85797,POINT (-73.89036 40.65520),1,0.124759


In [178]:
tpall

,geometry,subclass,NDVI
0,POINT (-73.95018 40.80481),1,0.061114
1,POINT (-73.95173 40.80921),2,-0.068959
2,POINT (-73.94942 40.80449),1,-0.001804
3,POINT (-73.95062 40.80500),2,0.038803
4,POINT (-73.95038 40.80490),2,0.227972
...,...,...,...
85794,POINT (-73.89033 40.65515),1,0.092050
85795,POINT (-73.90517 40.64284),6,0.148645
85796,POINT (-73.90518 40.64321),2,0.002555
85797,POINT (-73.89036 40.65520),1,0.124759


In [182]:
## tpall에서 NDVI에 0값 확인... 근데 다른 소숫점은 확인이 안됨
test = tpall[tpall['NDVI'] < 0.2]
test

,geometry,subclass,NDVI
0,POINT (-73.95018 40.80481),1,0.061114
1,POINT (-73.95173 40.80921),2,-0.068959
2,POINT (-73.94942 40.80449),1,-0.001804
3,POINT (-73.95062 40.80500),2,0.038803
5,POINT (-73.95074 40.80505),1,0.164838
...,...,...,...
85794,POINT (-73.89033 40.65515),1,0.092050
85795,POINT (-73.90517 40.64284),6,0.148645
85796,POINT (-73.90518 40.64321),2,0.002555
85797,POINT (-73.89036 40.65520),1,0.124759


## 이상치 제거: NDVI < 0.2 삭제

In [183]:
tp = tpall.drop(tpall[tpall['NDVI']<0.2].index)
tp

,geometry,subclass,NDVI
4,POINT (-73.95038 40.80490),2,0.227972
8,POINT (-73.94979 40.80465),1,0.220261
9,POINT (-73.94973 40.80462),4,0.293061
10,POINT (-73.94993 40.80471),1,0.464282
13,POINT (-73.94916 40.80438),1,0.414272
...,...,...,...
85771,POINT (-73.90318 40.66614),1,0.367431
85788,POINT (-73.88648 40.65577),1,0.542233
85789,POINT (-73.88751 40.65815),2,0.232292
85790,POINT (-73.88742 40.65818),2,0.452200


In [ ]:
NDVI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/NDVI.tif'
NDVI_src = rasterio.open(NDVI_file) # 파일 읽기
gdf[Path(NDVI_file).stem] = [x for x in NDVI_src.sample(coord_list)] # 포인트 샘플링
gdf[Path(NDVI_file).stem] = gdf[Path(NDVI_file).stem].astype('float64')
gdf.head()

In [ ]:
# ndvi 0.2 이하 point 제거
# 1. 포인트샘플링, 2. 제거

In [61]:
# NDVI 0.2 이하 point 제거 이후, 3by3 생성
def pointsampling(area, file, src, coord_list, feature):
    for i in range(0, 9):
        WIDTH = 0.00000599299783809013749338974087784241
        HEIGHT = 0.0000059299783809007439469768700121737
        coord_list0= [(x,y) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list1= [(x-WIDTH, y+HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list2= [(x, y+HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list3= [(x+WIDTH,y+HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list4= [(x+WIDTH,y) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list5= [(x+WIDTH,y-HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list6= [(x,y-HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list7= [(x-WIDTH,y-HEIGHT) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        coord_list8= [(x-WIDTH,y) for x,y in zip(tp02['geometry'].x , tp02['geometry'].y)]
        
        # 합치는 것만 생각해바
        tp02[Path(file).stem] = [x for x in src.sample(coord_list+'i')] # 포인트 샘플링
        tp02[feature] = tp02[feature].astype('float64') # 샘플링 값 dtype str to float64
        tp02[feature].drop(tp02[feature] == 0.0, inplace=True) # 그.. up, down, br을 합치고 싶은데... 어케할가,,
        tp02.rename(columns={feature: feautre+'_'+i}, inplace=True)
    tp02.head()
    print(len(tp02.index))
    return tp02

In [63]:
tpup = pointsampling(tp_up, NDVIup_file, NDVIdown_src, coord_list, "NDVI")

AttributeError: 'float' object has no attribute 'index'

In [ ]:
ndvi_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/manup2017/NDVI.tif'
ndvi_src = rasterio.open(ndvi_file) # 파일 읽기
tp[Path(ndvi_file).stem] = [x for x in ndvi_src.sample(coord_list)] # 포인트 샘플링
tp[Path(ndvi_file).stem] = tp[Path(ndvi_file).stem].astype('float64')

In [53]:
tp[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list)] # 포인트 샘플링
#tp[Path(NDVIup_file).stem] = tp[Path(NDVIup_file).stem].astype('float64')
tp[Path(NDVIup_file).stem]

0          [0.061114295761074494]
1          [-0.06895863107852732]
2        [-0.0018042394869449067]
3          [0.038802622168380596]
4           [0.22797190493318442]
                   ...           
85794                       [0.0]
85795                       [0.0]
85796                       [0.0]
85797                       [0.0]
85798                       [0.0]
Name: NDVI, Length: 85799, dtype: object

In [54]:
tp[Path(NDVIup_file).stem] = tp[Path(NDVIup_file).stem].astype('float64')
tp[Path(NDVIup_file).stem]

0        0.061114
1       -0.068959
2       -0.001804
3        0.038803
4        0.227972
           ...   
85794    0.000000
85795    0.000000
85796    0.000000
85797    0.000000
85798    0.000000
Name: NDVI, Length: 85799, dtype: float64

In [55]:
type(tp[Path(NDVIup_file).stem])

pandas.core.series.Series